In [1]:
import pandas as pd
import math
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm

In [ ]:
amf = pd.read_csv('data/dr9_main_complete.csv', header=None)
amf.columns = ['amf_no','amf_ra','amf_dec','amf_z','amf_lk','amf_rh','amf_r200','amf_rc','amf_c','amf_str_rh','bax_id','mcxc_id','red_id','whl_id']

c = SkyCoord(amf['amf_ra'],amf['amf_dec'],frame='icrs',unit='deg')
amf['glat'] = c.galactic.b.degree
amf['glon'] = c.galactic.l.degree
amf['cat'] = 0
amf_whl = amf[(amf.whl_id!='-9999')]

In [ ]:
amf

In [ ]:
hdul = fits.open("data/galaxy_clusters_desdr2.fits")
data = hdul[1].data
zou1 = pd.DataFrame({'ra':data['RA_PEAK'],
                    'dec':data['DEC_PEAK'],
                    'z':data['PHOTO_Z_PEAK'],
                    'rh':data['RICHNESS']})
c = SkyCoord(zou1['ra'],zou1['dec'],frame='icrs',unit='deg')
zou1['glat'] = c.galactic.b.degree
zou1['glon'] = c.galactic.l.degree
zou1['cat'] = 1

In [ ]:
hdul = fits.open("data/galaxy_clusters_desidr9.fits")
data = hdul[1].data
zou2 = pd.DataFrame({'ra':data['RA_PEAK'],
                    'dec':data['DEC_PEAK'],
                    'z':data['PHOTO_Z_PEAK'],
                    'rh':data['RICHNESS']})
c = SkyCoord(zou2['ra'],zou2['dec'],frame='icrs',unit='deg')
zou2['glat'] = c.galactic.b.degree
zou2['glon'] = c.galactic.l.degree
zou2['cat'] = 2

In [ ]:
hdul = fits.open("data/galaxy_clusters_hscpdr3_wide.fits")
data = hdul[1].data
zou3 = pd.DataFrame({'ra':data['RA_PEAK'],
                    'dec':data['DEC_PEAK'],
                    'z':data['PHOTO_Z_PEAK'],
                    'rh':data['RICHNESS']})
c = SkyCoord(zou3['ra'],zou3['dec'],frame='icrs',unit='deg')
zou3['glat'] = c.galactic.b.degree
zou3['glon'] = c.galactic.l.degree
zou3['cat'] = 3

In [ ]:
zou = pd.concat([zou1,zou2,zou3])

In [ ]:
# Mass in 10^14
wh22 = pd.read_csv('data/cluster_DESunWISE.dat', sep='\s+', header=None, usecols=[3,4,5,10,11])
wh22.columns = ['ra','dec','z','rh','m']
c = SkyCoord(wh22['ra'],wh22['dec'],frame='icrs',unit='deg')
wh22['glat'] = c.galactic.b.degree
wh22['glon'] = c.galactic.l.degree
wh22['cat'] = 4

In [ ]:
# dcl.z = np.array(dcl.z).byteswap().newbyteorder()

In [ ]:
amf0 = amf.rename(columns={'amf_ra':'amf_ra','amf_dec':'amf_dec','amf_z':'z','amf_rh':'rh'})[['amf_ra','amf_dec','z','rh','cat']]
zou0 = zou.rename(columns={'ra':'amf_ra','dec':'amf_dec'})[['amf_ra','amf_dec','z','rh','cat']]
wh220 = wh22.rename(columns={'ra':'amf_ra','dec':'amf_dec'})[['amf_ra','amf_dec','z','rh','cat']]

union = pd.concat([amf0,zou0,wh220])#.to_csv('../../data/union.csv')

In [ ]:
union

## Nearest Neighbors ##

### DES, WH22 Intersection ###

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming you already have a DataFrame named 'df' with columns 'x' and 'y'

cut = pd.read_csv("data/union3.csv", skiprows = 0)
des = cut[cut.cat == 1]
wh22 = cut[cut.cat == 4]
des_wh22 = cut[cut.cat.isin([1,4])].reset_index(drop = True)

des_wh22

In [ ]:
coords = des_wh22[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)
indices

In [ ]:
# Check if there's any point within 0.5 distance for each row, excluding the point itself (hence k=2)
# We can do this by checking if the second smallest distance is <= 0.5

# To get intersection, not XOR
des_wh22['Match'] = distances[:,1] <= 1e-2
des_wh22 = des_wh22.reset_index(drop = True)

dupes = pd.Series(indices[:,1])
dupes = dupes[dupes != len(des_wh22)]
dupes = dupes[dupes < len(des)]
print(dupes)

print(des_wh22)
# changes dupes to dupes.index to go back and forth
des_wh22 = des_wh22.drop(dupes.index, axis = "index")
des_wh22 = des_wh22[des_wh22["Match"]]

des_wh22

In [ ]:
des_wh22 = des_wh22.reset_index(drop = True)
des_wh22 = des_wh22.drop("Match", axis = "columns")
# des_wh22.to_csv("output/Locations/des_wh22_AND_1.csv", index = False)
des_wh22

In [ ]:
# Only check to make sure nothings in the intersection anymore

coords = des_wh22[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)
check = pd.Series(indices[:,1])
check = check[check != len(des_wh22)]
des_wh22

### Finding DES, WH22 and DESI Intersection ###

In [166]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Check for intersection of above, AND DESI as well.

cut = pd.read_csv("data/union3.csv", skiprows = 0)

des = cut[cut.cat == 1]
desi = cut[cut.cat == 2]
des_wh22 = pd.read_csv("output/Locations/des_wh22_AND_1.csv", skiprows = 0)

# If in the redshift range needed (I'm making it generalized for now, so running without it):

# desi = desi[(desi.z > 0.6) & (desi.z < 1)].reset_index(drop = True)
# des_wh22 = des_wh22[(des_wh22.z > 0.6) & (des_wh22.z < 1)].reset_index(drop = True)


# Remove duplicates in DESI (also saved below)

desi_coords = desi[['amf_ra', 'amf_dec','z']].values
desi_tree = KDTree(desi_coords)
desi_distances, desi_indices = desi_tree.query(desi_coords, k=2, distance_upper_bound=1e-2)

desi_dupes = pd.Series(desi_indices[:,1], index = desi.index)
desi_dupes = desi_dupes[desi_dupes != len(desi)]

desi = desi.drop(desi_dupes.index, axis = "index")

des_wh22_desi = pd.concat([des_wh22, desi])
des_wh22_desi

,amf_ra,amf_dec,z,rh,cat
0,0.790352,5.325782,0.245052,112.881846,1
1,12.665345,4.895529,0.498291,127.757842,1
2,26.601518,4.943122,0.651660,134.416565,1
3,32.490271,4.958005,0.609625,151.064995,1
4,35.356838,5.058750,0.823470,132.762191,1
...,...,...,...,...,...
666247,61.088736,-67.972281,0.440988,53.872143,2
666248,61.212453,-67.932890,0.462506,127.688333,2
666249,61.344614,-67.881257,0.419552,101.967931,2
666250,61.468347,-67.935127,0.480716,39.670418,2


In [161]:
# Now find intersection of DES, WH22, and DESI

coords = des_wh22_desi[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)
indices

array([[     0, 534823],
       [     1, 534823],
       [     2, 534823],
       ...,
       [534820, 534823],
       [534821, 534823],
       [534822, 534823]])

In [162]:
# Check if there's any point within 0.5 distance for each row, excluding the point itself (hence k=2)
# We can do this by checking if the second smallest distance is <= 0.5

# To get intersection, not XOR
des_wh22_desi['Match'] = distances[:,1] <= 1e-2

dupes = pd.Series(indices[:,1], index = des_wh22_desi.index)
dupes = dupes[dupes != len(des_wh22_desi)]
dupes = dupes[dupes < len(des_wh22)]
print(dupes)

print(des_wh22_desi)
des_wh22_desi = des_wh22_desi.drop(dupes, axis = "index")
des_wh22_desi = des_wh22_desi[des_wh22_desi["Match"]]

test = des_wh22_desi
des_wh22_desi

415587       5
418624       6
426532      27
426626      31
426792      34
          ... 
662568    2011
663492    2018
663670    2022
663744    2023
665374    2043
Length: 282, dtype: int64
           amf_ra    amf_dec         z          rh  cat  Match
0        0.790352   5.325782  0.245052  112.881846    1  False
1       12.665345   4.895529  0.498291  127.757842    1  False
2       26.601518   4.943122  0.651660  134.416565    1  False
3       32.490271   4.958005  0.609625  151.064995    1  False
4       35.356838   5.058750  0.823470  132.762191    1  False
...           ...        ...       ...         ...  ...    ...
666247  61.088736 -67.972281  0.440988   53.872143    2  False
666248  61.212453 -67.932890  0.462506  127.688333    2  False
666249  61.344614 -67.881257  0.419552  101.967931    2  False
666250  61.468347 -67.935127  0.480716   39.670418    2  False
666251  60.051419 -68.070351  0.413162   61.881470    2  False

[534823 rows x 6 columns]


,amf_ra,amf_dec,z,rh,cat,Match
415587,35.099632,4.845692,0.320425,112.168702,2,True
418624,6.703375,4.642850,0.778930,151.347998,2,True
426532,5.020381,3.927658,0.712172,116.505035,2,True
426626,10.942642,3.471896,0.143705,88.328109,2,True
426792,20.807051,3.716050,0.611859,183.965980,2,True
...,...,...,...,...,...,...
662568,310.424194,-63.662801,0.519585,107.391509,2,True
663492,45.885513,-64.197486,0.657558,116.523830,2,True
663670,62.997521,-64.606550,0.157728,143.517970,2,True
663744,66.943806,-64.184260,0.616502,203.810259,2,True


In [164]:
modified = test.drop("Match", axis = "columns").reset_index()
total = pd.read_csv("output/Locations/des_wh22_desi_AND_2.csv", skiprows = 0)
# modified[modified.cat != 2]
print(total)
print(modified)
result = modified.merge(total, how = "inner")
result

indic = result["index"]
indic

         amf_ra    amf_dec         z          rh  cat
0     35.099632   4.845692  0.320425  112.168702    2
1      6.703375   4.642850  0.778930  151.347998    2
2      5.020381   3.927658  0.712172  116.505035    2
3     10.942642   3.471896  0.143705   88.328109    2
4     24.600070   3.032973  0.251419   79.509373    2
..          ...        ...       ...         ...  ...
186    0.811760 -61.849984  0.523437  126.112668    2
187   42.849088 -61.949780  0.542307  258.624965    2
188  320.540248 -62.127096  0.708371  138.423358    2
189   45.885513 -64.197486  0.657558  116.523830    2
190   62.997521 -64.606550  0.157728  143.517970    2

[191 rows x 5 columns]
      index      amf_ra    amf_dec         z          rh  cat
0    415587   35.099632   4.845692  0.320425  112.168702    2
1    418624    6.703375   4.642850  0.778930  151.347998    2
2    426532    5.020381   3.927658  0.712172  116.505035    2
3    426626   10.942642   3.471896  0.143705   88.328109    2
4    426792   20.8

0      415587
1      418624
2      426532
3      426626
4      426881
        ...  
186    658771
187    659203
188    659884
189    663492
190    663670
Name: index, Length: 191, dtype: int64

In [165]:
jam = dupes.loc[indic]
jam

415587       5
418624       6
426532      27
426626      31
426881      37
          ... 
658771    1971
659203    1976
659884    1985
663492    2018
663670    2022
Length: 191, dtype: int64

In [51]:
result = test1.merge(test2, how = "inner", left_index=True, right_index=True)
result

,amf_ra_x,amf_dec_x,z_x,rh_x,cat_x,Match_x,amf_ra_y,amf_dec_y,z_y,rh_y,cat_y,Match_y
415587,35.099632,4.845692,0.320425,112.168702,2,True,35.099632,4.845692,0.320425,112.168702,2,True
418624,6.703375,4.642850,0.778930,151.347998,2,True,6.703375,4.642850,0.778930,151.347998,2,True
426532,5.020381,3.927658,0.712172,116.505035,2,True,5.020381,3.927658,0.712172,116.505035,2,True
426626,10.942642,3.471896,0.143705,88.328109,2,True,10.942642,3.471896,0.143705,88.328109,2,True
426881,24.600070,3.032973,0.251419,79.509373,2,True,24.600070,3.032973,0.251419,79.509373,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
658771,0.811760,-61.849984,0.523437,126.112668,2,True,0.811760,-61.849984,0.523437,126.112668,2,True
659203,42.849088,-61.949780,0.542307,258.624965,2,True,42.849088,-61.949780,0.542307,258.624965,2,True
659884,320.540248,-62.127096,0.708371,138.423358,2,True,320.540248,-62.127096,0.708371,138.423358,2,True
663492,45.885513,-64.197486,0.657558,116.523830,2,True,45.885513,-64.197486,0.657558,116.523830,2,True


In [55]:
des_wh22_desi = des_wh22_desi.reset_index(drop = True)
# des_wh22_desi = des_wh22_desi.drop("Match", axis = "columns")
# des_wh22_desi.to_csv("output/Locations/des_wh22_desi_AND_2.csv", index = False)
des_wh22_desi

,amf_ra,amf_dec,z,rh,cat
0,35.099632,4.845692,0.320425,112.168702,2
1,6.703375,4.642850,0.778930,151.347998,2
2,5.020381,3.927658,0.712172,116.505035,2
3,10.942642,3.471896,0.143705,88.328109,2
4,24.600070,3.032973,0.251419,79.509373,2
...,...,...,...,...,...
192,0.811760,-61.849984,0.523437,126.112668,2
193,42.849088,-61.949780,0.542307,258.624965,2
194,320.540248,-62.127096,0.708371,138.423358,2
195,45.885513,-64.197486,0.657558,116.523830,2


#### Checking DES, WH22, DESI for duplicates ####

In [ ]:
# Remove duplicates in DES

des_coords = des[['amf_ra', 'amf_dec','z']].values
des_tree = KDTree(des_coords)
des_distances, des_indices = des_tree.query(des_coords, k=2, distance_upper_bound=1e-2)

des['Match'] = des_distances[:,1] <= 1e-2
des = des.reset_index(drop = True)

des_dupes = pd.Series(des_indices[:,1])
des_dupes = des_dupes[des_dupes != len(des)]

des = des.drop(des_dupes.index, axis = "index")
des

# END RESULT --> there are no duplicates in DES.

In [ ]:
# Remove duplicates in WH22

wh22_coords = wh22[['amf_ra', 'amf_dec','z']].values
wh22_tree = KDTree(wh22_coords)
wh22_distances, wh22_indices = wh22_tree.query(wh22_coords, k=2, distance_upper_bound=1e-2)

wh22['Match'] = wh22_distances[:,1] <= 1e-2
wh22 = wh22.reset_index(drop = True)

wh22_dupes = pd.Series(wh22_indices[:,1])
wh22_dupes = wh22_dupes[wh22_dupes != len(wh22)]

wh22 = wh22.drop(wh22_dupes.index, axis = "index")
wh22

# END RESULT --> there are no duplicates in WH22.

In [ ]:
# Remove duplicates in DESI

desi_coords = desi[['amf_ra', 'amf_dec','z']].values
desi_tree = KDTree(desi_coords)
desi_distances, desi_indices = desi_tree.query(desi_coords, k=2, distance_upper_bound=1e-2)

desi['Match'] = desi_distances[:,1] <= 1e-2
desi = desi.reset_index(drop = True)

desi_dupes = pd.Series(desi_indices[:,1])
desi_dupes = desi_dupes[desi_dupes != len(desi)]

desi = desi.drop(desi_dupes.index, axis = "index")
desi

# END RESULT --> there are SOME duplicates in DESI.

## Universal Reference ##

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming you already have a DataFrame named 'df' with columns 'x' and 'y'
df = union.copy()
coords = df[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)

# Check if there's any point within 0.5 distance for each row, excluding the point itself (hence k=2)
# We can do this by checking if the second smallest distance is <= 0.5
df['Match'] = distances[:, 1] <= 1e-2

In [ ]:
df[df['Match']].round(1).drop_duplicates(subset=['amf_ra','amf_dec','z'])

In [ ]:
deswh = union[(union.cat.isin([1,4]))]

import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming you already have a DataFrame named 'deswh' with columns 'x' and 'y'
coords = deswh[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)

# Check if there's any point within 0.5 distance for each row, excluding the point itself (hence k=2)
# We can do this by checking if the second smallest distance is <= 0.5
deswh['Match'] = distances[:, 1] <= 1e-2

In [ ]:
deswh[deswh['Match']].round(1).drop_duplicates(subset=['amf_ra','amf_dec','z'])

In [ ]:
hscwh = union[(union.cat.isin([3,4]))]

import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming you already have a DataFrame named 'hscwh' with columns 'x' and 'y'
coords = hscwh[['amf_ra', 'amf_dec','z']].values

# Create a k-d tree from the coordinates
tree = KDTree(coords)

# Query the k-d tree for nearest neighbors within 0.5 distance
# Note that each point will find itself as the nearest neighbor with distance 0
distances, indices = tree.query(coords, k=2, distance_upper_bound=1e-2)

# Check if there's any point within 0.5 distance for each row, excluding the point itself (hence k=2)
# We can do this by checking if the second smallest distance is <= 0.5
hscwh['Match'] = distances[:, 1] <= 1e-2

In [ ]:
hscwh[hscwh['Match']].round(1).drop_duplicates(subset=['amf_ra','amf_dec','z'])